In [2]:
import gzip
import json
from typing import Any, Iterator

import pandas as pd

# Table of Contents

- [Step A.1](#Step-A.1)
- [Step A.2](#Step-A.2)

In [ ]:
# download the “small” 5-core dataset for the category "Digital Music"
# dataset source: https://nijianmo.github.io/amazon/index.html

!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Digital_Music_5.json.gz -P data/

In [ ]:
# download the metadata for this dataset
# dataset source: https://nijianmo.github.io/amazon/index.html

!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Digital_Music.json.gz -P data/

## Step A.1

The 5-core dataset for the category "Digital Music" subset of the [Amazon Review data](https://nijianmo.github.io/amazon/index.html) in which all users and items have at least 5 reviews.

The format is one-review-per-line in JSON, with the following attributes:

- `reviewerID` - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- `asin` - ID of the product, e.g. 0000013714
- `reviewerName` - name of the reviewer
- `vote` - helpful votes of the review
- `style` - a disctionary of the product metadata, e.g., "Format" is "Hardcover"
- `reviewText` - text of the review
- `overall` - rating of the product
- `summary` - summary of the review
- `unixReviewTime` - time of the review (unix time)
- `reviewTime` - time of the review (raw)
- `image` - images that users post after they have received the product

Metadata includes descriptions, price, sales-rank, brand info, and co-purchasing links:

- `asin` - ID of the product, e.g. 0000031852
- `title` - name of the product
- `feature` - bullet-point format features of the product
- `description` - description of the product
- `price` - price in US dollars (at time of crawl)
- `imageURL` - url of the product image
- `imageURLHighRes` - url of the high resolution product image
- `related` - related products (also bought, also viewed, bought together, buy after viewing)
- `salesRank` - sales rank information
- `brand` - brand name
- `categories` - list of categories the product belongs to
- `tech1` - the first technical detail table of the product
- `tech2` - the second technical detail table of the product
- `similar` - similar product table

In [3]:
def inspect_df(df: pd.DataFrame, n: int = 5) -> pd.DataFrame:
    """Helper method to easily inspect DataFrames."""

    print(f"shape: {df.shape}")

    return df.head(n)

In [4]:
def parse(filepath: str) -> Iterator[dict]:
    file_obj = gzip.open(filepath, "rb")
    for line in file_obj:
        yield json.loads(line)

In [5]:
def file_to_dataframe(filepath: str) -> pd.DataFrame:
    i = 0
    df = {}
    for d in parse(filepath):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient="index")

In [6]:
review_data = file_to_dataframe("data/Digital_Music_5.json.gz")

inspect_df(review_data)

shape: (169781, 12)


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,3,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,1370217600,NaN
1,5.0,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,1412985600,NaN
2,5.0,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,1392076800,NaN
3,4.0,3,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,1386374400,NaN
4,5.0,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",1465689600,NaN


In [7]:
review_data.loc[2]

overall                                                         5.0
vote                                                            NaN
verified                                                       True
reviewTime                                              02 11, 2014
reviewerID                                           A2VAMODP8M77NG
asin                                                     3426958910
style                                      {'Format:': ' Audio CD'}
reviewerName                                                 JTGabq
reviewText        It was great to hear the old stuff again and I...
summary                                 SLAYER!!!!!!!!!!!!!!!!!!!!!
unixReviewTime                                           1392076800
image                                                           NaN
Name: 2, dtype: object

In [8]:
metadata = file_to_dataframe("data/meta_Digital_Music.json.gz")

inspect_df(metadata)

shape: (74347, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,[],[],NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,[],[],NaN
2,[],,[],,Early Works - Don Francisco,"[B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1Z...",,Don Francisco,[],"875,825 in CDs & Vinyl (","[B003H8F4NA, B003ZFVHPO, B003JMP1ZK, B00004RC0...","<img src=""https://images-na.ssl-images-amazon....",,,,0001527134,[],[],NaN
3,[],,[],,So You Wanna Go Back to Egypt,"[B0000275QQ, 0001393774, 0001388312, B0016CP2G...",,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,0001388703,[],[],NaN
4,[],,[1. Losing Game 2. I Can't Wait 3. Didn't He S...,,Early Works - Dallas Holm,"[B0002N4JP2, 0760131694, B00002EQ79, B00150K8J...",,Dallas Holm,[],"399,269 in CDs & Vinyl (","[B0002N4JP2, 0760131694, B00150K8JC, B003MTXNV...","<img src=""https://images-na.ssl-images-amazon....",,,,0001526146,[],[],NaN


In [9]:
metadata[metadata["asin"] == review_data.loc[2]["asin"]]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
343,[],,[],,Slayer - Greatest Hits 2 CD Set,[],,Slayer,[],"231,252 in CDs &amp; Vinyl (","[B00EBJ62GC, B00BV0W7ZE, B000062YAY, B00EBWHW5...","<img src=""https://images-na.ssl-images-amazon....",,,,3426958910,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [ ]:
review_data["asin"] = review_data["asin"].map(str.strip)
metadata["asin"] = metadata["asin"].map(str.strip)

In [32]:
data = pd.merge(review_data, metadata, how="left", on="asin")

inspect_df(data)

shape: (169829, 30)


,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,feature,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details
0,5.0,3,True,"06 3, 2013",A2TYZ821XXK2YZ,3426958910,{'Format:': ' Audio CD'},Garrett,"This is awesome to listen to, A must-have for ...",Slayer Rules!,...,[],"231,252 in CDs &amp; Vinyl (","[B00EBJ62GC, B00BV0W7ZE, B000062YAY, B00EBWHW5...","<img src=""https://images-na.ssl-images-amazon....",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,5.0,NaN,True,"10 11, 2014",A3OFSREZADFUDY,3426958910,{'Format:': ' Audio CD'},Ad,bien,Five Stars,...,[],"231,252 in CDs &amp; Vinyl (","[B00EBJ62GC, B00BV0W7ZE, B000062YAY, B00EBWHW5...","<img src=""https://images-na.ssl-images-amazon....",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,5.0,NaN,True,"02 11, 2014",A2VAMODP8M77NG,3426958910,{'Format:': ' Audio CD'},JTGabq,It was great to hear the old stuff again and I...,SLAYER!!!!!!!!!!!!!!!!!!!!!,...,[],"231,252 in CDs &amp; Vinyl (","[B00EBJ62GC, B00BV0W7ZE, B000062YAY, B00EBWHW5...","<img src=""https://images-na.ssl-images-amazon....",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,4.0,3,False,"12 7, 2013",AAKSLZ9IDTEH0,3426958910,{'Format:': ' Audio CD'},john F&#039;n doe,well best of's are a bit poison normally but t...,slayer greatest hits! you mean everything righ...,...,[],"231,252 in CDs &amp; Vinyl (","[B00EBJ62GC, B00BV0W7ZE, B000062YAY, B00EBWHW5...","<img src=""https://images-na.ssl-images-amazon....",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,5.0,NaN,True,"06 12, 2016",A3OH43OZJLKI09,5557706259,{'Format:': ' Audio CD'},melinda a goodman,What can I say? This is Casting Crowns!!!This ...,"This is a good, blessing filled",...,[],"500,020 in CDs & Vinyl (","[B01I4I5XBK, B016W643JY, B000AA7HGK, B0000CDL6...","<img src=""https://images-na.ssl-images-amazon....",,,$16.98,[],[],NaN


## Step A.2